# Enricher Basics

 Welcome to the Enricher module 👋 where we’ll sprinkle some extra magic onto your urban layers. Let’s add average building floors to a layer and see it sparkle!

**What you’ll learn**:
- Loading data and creating a layer.
- Enriching it with average floors.
- Previewing the enrichment setup.

Let’s jazz things up! 🏙️

In [ ]:
import urban_mapper as um

# Start UrbanMapper
mapper = um.UrbanMapper()

## Loading Data and Creating a Layer

First, let’s grab some PLUTO data and set up a street intersections layer for Downtown Brooklyn.

Note that:

- Loader example can be seen in `examples/Basics/loader.ipynb`
- Urban Layer example can be seen in `examples/Basics/urban_layer.ipynb`
- Imputer example can be seen in `examples/Basics/imputer.ipynb`

In [ ]:
# Load data
data = (
    mapper.loader.from_file(  # From the loader module
        "./pluto.csv"
    )  # Pluto data usually gives lot tax building information in NYC.
    .with_columns("longitude", "latitude")
    .load()
)

# Create urban layer
layer = (
    mapper.urban_layer.with_type("streets_intersections")  # From the urban_layer module
    .from_place("Downtown Brooklyn, New York City, USA")
    .build()
)

# Impute your data if they contain missing values
data = (
    mapper.imputer.with_type("SimpleGeoImputer")  # From the imputer module
    .on_columns("longitude", "latitude")
    .transform(
        data, layer
    )  # All imputers require access to the urban layer in case they need to extract information from it.
)

## Enriching the Layer

Now, let’s enrich the layer with the average number of floors per intersection. We’ll map the data, set up the enricher, and apply it.

More enriching workflows exist and would be available per the documentation. Hint: `count_by`


In [ ]:
# Map data to the nearest layer
# Here the point is to say which intersection of the city maps with which record(s) in your data
# so that we can take into account when enriching.
_, mapped_data = layer.map_nearest_layer(
    data,
    longitude_column="longitude",
    latitude_column="latitude",
    output_column="nearest_intersection",
)

# Set up and apply enricher
enricher = (
    mapper.enricher.with_data(  # From the enricher module
        group_by="nearest_intersection", values_from="numfloors"
    )
    # Reading: With data grouped by the nearest intersection, and the values from the attribute numfloors
    .aggregate_by(method="mean", output_column="avg_floors")
    # Reading: Aggregate by using the mean and output this into the avg_floors new attribute of the urban layer
    .build()
)
enriched_layer = enricher.enrich(
    mapped_data, layer
)  # Data to use, Urban Layer to Enrich.

## Be Able To Preview Your Enricher

Fancy a peek at your enricher? Use `preview()` to see the setup—great for when you’re digging into someone else’s work!

In [ ]:
# Preview enricher
print(enricher.preview())

## Wrapping Up

Smashing work! 🎉 Your layer’s now enriched with average floors. Try visualising it next with `visualiser`.